# Recommendation systems
Welcome to your first homework assignment about recommendation systems.


## The Cold Start Problem 

The colaborative filtering method discussed in class does not address the problem of new user or new movies. What prediction would you use in these cases:

* A new user but a known movie
* A new movie and a known user
* A new user and new movie

YOUR ANSWER HERE

## Matrix Factorization with bias
We want to extend the Matrix Factorization model discussed in class to add a "bias" parameter for each user and another "bias" parameter for each movie.  For the problem in class we had the parameters matrix $U$ and $V$, we are adding $u^0$ which is a vector of dimension $n_u$ and $v^0$ which is a vector of dimension $n_m$. The equations

$$\hat{y}_{ij} = u_{0i} + v_{0j} + u_i \cdot v_j  $$ 
 
(a) How many weights (parameters) are we fitting for this problem?

(b) Write the gradient descent equations for this problem.

YOUR ANSWER HERE

# Collaborative Filtering with Gradient Descent 

In this part of the assignment you will build a collaborative filtering model to predict netflix ratings.  This assignment will step you through how to do this using stochastic gradient descent. The data for this assignment is ...

**Instructions:**
- Do not use loops (for/while) in your code, unless the instructions explicitly ask you to do so.
- DO NOT change paths (-3 points)
- DO NOT submit data to github (-2 points)

**You will learn to:**
- Build the general architecture of a learning algorithm, including:
    - Encoding rating data
    - Initializing parameters
    - Calculating the cost function
    - Calculating gradient
    - Using an optimization algorithm (gradient descent) 
    - Predicting on new data
- Putting it all together.

In [ ]:
import numpy as np
import pandas as pd

## Encoding rating data
Here are our very small subset of fake data to get us started.

In [ ]:
# The first row says that user 1 reated movie 11 with a score of 4
!cat tiny_training2.csv 

In [ ]:
# here is a handy function from fast.ai
def proc_col(col):
    """Encodes a pandas column with continous ids. 
    """
    uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx[x] for x in col]), len(uniq)

In [ ]:
def encode_data(df):
    """Encodes rating data with continous user and movie ids using 
    the helpful fast.ai function from above.
    
    Arguments:
      train_csv: a csv file with columns user_id,movie_id,rating 
    
    Returns:
      df: a dataframe with the encode data
      num_users
      num_movies
      
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return df, num_users, num_movies

In [ ]:
df = pd.read_csv("tiny_training2.csv")
df, num_users, num_movies = encode_data(df)

In [ ]:
df

In [ ]:
assert(num_users == 7)

In [ ]:
assert(num_movies == 4)

In [ ]:
np.testing.assert_equal(df["userId"].values, np.array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 6, 6]))

## Initializing parameters

In [ ]:
def create_embedings(n, K):
    """ Create a numpy random matrix of shape n, K
    
    The random matrix should be initialized with uniform values in (0, 6/K)
    Arguments:
    
    Inputs:
    n: number of items/users
    K: number of factors in the embeding 
    
    Returns:
    emb: numpy array of shape (n, num_factors)
    """
    np.random.seed(3)
    emb = 6*np.random.random((n, K)) / K
    return emb

# here is an example on how the prediction matrix would look like with 7 users and 5 movies
np.dot(create_embedings(7,3), create_embedings(5,3).transpose())

## Encoding Y as a sparse matrix
This code helps you encode a $Y$ as a sparse matrix from the dataframe. 

In [ ]:
from scipy import sparse
def df2matrix(df, nrows, ncols, column_name="rating"):
    """ Returns a sparse matrix constructed from a dataframe
    
    This code assumes the df has columns: MovieID,UserID,Rating
    """
    values = df[column_name].values
    ind_movie = df['movieId'].values
    ind_user = df['userId'].values
    return sparse.csc_matrix((values,(ind_user, ind_movie)),shape=(nrows, ncols))

In [ ]:
df = pd.read_csv("tiny_training2.csv")
df, num_users, num_movies = encode_data(df)
Y = df2matrix(df, num_users, num_movies)

In [ ]:
print(Y)

In [ ]:
def sparse_multiply(df, emb_user, emb_movie):
    """ This function returns U*V^T element wise multi by R as a sparse matrix.
    
    It avoids creating the dense matrix U*V^T
    """
    df["Prediction"] = np.sum(emb_user[df["userId"].values]*emb_movie[df["movieId"].values], axis=1)
    return df2matrix(df, emb_user.shape[0], emb_movie.shape[0], column_name="Prediction")

## Calculating the cost function

In [ ]:
# Use vectorized computation for this function. No loops!
# Hint: use df2matrix and sparse_multiply
def cost(df, emb_user, emb_movie):
    """ Computes mean square error
    
    First compute prediction. Prediction for user i and movie j is
    emb_user[i]*emb_movie[j]
    
    Arguments:
      df: dataframe with all data or a subset of the data
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      error(float): this is the MSE
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return error

In [ ]:
emb_user = np.ones((num_users, 3))
emb_movie = np.ones((num_movies, 3))
error = cost(df, emb_user, emb_movie)
assert(np.around(error, decimals=2) == 2.23)

## Calculating gradient

In [ ]:
def finite_difference(df, emb_user, emb_movie, ind_u=None, ind_m=None, k=None):
    """ Computes finite difference on MSE(U, V).
    
    This function is used for testing the gradient function. 
    """
    e = 0.000000001
    c1 = cost(df, emb_user, emb_movie)
    K = emb_user.shape[1]
    x = np.zeros_like(emb_user)
    y = np.zeros_like(emb_movie)
    if ind_u is not None:
        x[ind_u][k] = e
    else:
        y[ind_m][k] = e
    c2 = cost(df, emb_user + x, emb_movie + y)
    return (c2 - c1)/e

In [ ]:
def gradient(df, Y, emb_user, emb_movie):
    """ Computes the gradient.
    
    First compute prediction. Prediction for user i and movie j is
    emb_user[i]*emb_movie[j]
    
    Arguments:
      df: dataframe with all data or a subset of the data
      Y: sparse representation of df
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      d_emb_user
      d_emb_movie
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
K = 3
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
grad_user, grad_movie = gradient(df, Y, emb_user, emb_movie)

In [ ]:
user=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_u=user, k=i) for i in range(K)])
assert(np.all(np.abs(grad_user[user] - approx) < 0.0001))

In [ ]:
movie=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_m=movie, k=i) for i in range(K)])
assert(np.all(np.abs(grad_movie[movie] - approx) < 0.0001))

## Using gradient descent with momentum

In [ ]:
# you can use a for loop to iterate through gradient descent
def gradient_descent(df, emb_user, emb_movie, iterations=100, learning_rate=0.01, df_val=None):
    """ Computes gradient descent with momentum (0.9) for a number of iterations.
    
    Prints training cost and validation cost (if df_val is not None) every 50 iterations.
    
    Returns:
    emb_user: the trained user embedding
    emb_movie: the trained movie embedding
    """
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    # YOUR CODE HERE
    raise NotImplementedError()
    return emb_user, emb_movie

In [ ]:
emb_user = create_embedings(num_users, 3)
emb_movie = create_embedings(num_movies, 3)
emb_user, emb_movie = gradient_descent(df, emb_user, emb_movie, iterations=200, learning_rate=0.01)

In [ ]:
train_mse = cost(df, emb_user, emb_movie)
assert(np.around(train_mse, decimals=2) == 0.53)

## Predicting on new data
Now we should write a function that given new data is able to predict ratings. First we write a function that encodes new data. If a new user or item is present that row should be remove. Collaborative Filtering is not good at handling new users or new items. To help with this task, you could write a an auxiliary function similar to `proc_col`.

In [ ]:
def encode_new_data(df_val, df_train):
    """ Encodes df_val with the same encoding as df_train.
    Returns:
    df_val: dataframe with the same encoding as df_train
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return df_val

In [ ]:
df_t = pd.read_csv("tiny_training2.csv")
df_v = pd.read_csv("tiny_val2.csv")
df_v = encode_new_data(df_v, df_t)

In [ ]:
assert(len(df_v.userId.unique())==2)

In [ ]:
assert(len(df_v) == 2)

## Putting it all together
For this part you should get data from here
`wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip`

In [ ]:
# Don't change this path use a simlink if you have the data somewhere else
path = "ml-latest-small/"
data = pd.read_csv(path + "ratings.csv")
# sorting by timestamp take as validation data the most recent data doesn't work so let's just take 20%
# at random
np.random.seed(3)
msk = np.random.rand(len(data)) < 0.8
train = data[msk].copy()
val = data[~msk].copy()
df_train, num_users, num_movies = encode_data(train.copy())
df_val = encode_new_data(val.copy(), train.copy())
print(len(val), len(df_val))

In [ ]:
K = 50
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
emb_user, emb_movie = gradient_descent(df_train, emb_user, emb_movie, iterations=2000, learning_rate=1, df_val=df_val)

In [ ]:
train_mse = cost(df_train, emb_user, emb_movie)
val_mse = cost(df_val, emb_user, emb_movie)
print(train_mse, val_mse)

In [ ]:
train_mse = cost(df_train, emb_user, emb_movie)
assert(np.around(train_mse, decimals=2) == 0.77)

In [ ]:
val_mse = cost(df_val, emb_user, emb_movie)
assert(np.around(val_mse, decimals=2) == 0.91)

#  Advanced Recommendation (Optional)
Use Regression / Random forest to add other features to your recommendation algorithm. Here are the steps:

* Here are potential features: 
    * user embeding, movie embeding (from the hw)
    * movie genres
    * daypart features
    
Other extentions: add regularization to gradient descent

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()